<a href="https://colab.research.google.com/github/neal-logan/dsba6211-summer2024/blob/main/nophishing/01_exploratory_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 01 Exploratory Analysis


### Setup

#### Required Packages

In [ ]:
# Developed with Python version 3.10.12 in Colab version 1.0.0

#https://github.com/SelfExplainML/PiML-Toolbox
!pip install PiML==0.6.0
#https://github.com/facebookresearch/hiplot
!pip install hiplot==0.1.33




#### Random Seed

In [37]:
random_seed = 42

#### Load and Prepare Data

In [32]:
# Load and prepare training data
import pandas as pd

train_url = 'https://raw.githubusercontent.com/neal-logan/dsba6211-summer2024/main/nophishing/data/phishing-url-pirochet-train.csv'
df = pd.read_csv(train_url)

#Create numeric target variable column
df['y'] = df['status'].replace('legitimate', 0).replace('phishing', 1)

#Drop unnecessary columns
df = df.drop(columns=['status','url'])

#X/y split
X = df.drop(columns=['y'])
y = df['y']

In [9]:
#Split training set into training and validation set (test set not yet loaded)

from sklearn.model_selection import train_test_split
X_train, X_validation, y_train, y_validation = train_test_split(
    X,
    y,
    test_size = 0.2,
    random_state = random_seed)



### Overview of the Data

Most of the columns are a mix of binary, small discrete numbers, or ratios in decimal format.  Some columns have significantly larger values.  Some columns contain negative values that are apparently invalid.

While the column names follow some degree of convention, there's no simple way to delineate how each of them should be handled.  I will need to analyze each feature individually and organize a preprocessing pipeline that takes into account what each feature needs.

Some of the fields are obtained from third-party providers.  Because these might not always be available, I will develop a set of models that use this data as well as a second set of models that do not.

#### Categorize Features by Source

In [33]:
# Split features into categories based on origin: from URL, from site content,
# or from third parties

url_columns = ['length_url', 'length_hostname', 'ip', 'nb_dots',
  'nb_hyphens', 'nb_at', 'nb_qm', 'nb_and', 'nb_or', 'nb_eq',
  'nb_underscore', 'nb_tilde', 'nb_percent', 'nb_slash', 'nb_star',
  'nb_colon', 'nb_comma', 'nb_semicolumn', 'nb_dollar', 'nb_space',
  'nb_www', 'nb_com', 'nb_dslash', 'http_in_path', 'https_token',
  'ratio_digits_url', 'ratio_digits_host', 'punycode', 'port',
  'tld_in_path', 'tld_in_subdomain', 'abnormal_subdomain',
  'nb_subdomains', 'prefix_suffix', 'random_domain', 'shortening_service',
  'path_extension', 'nb_redirection', 'nb_external_redirection',
  'length_words_raw', 'char_repeat', 'shortest_words_raw',
  'shortest_word_host', 'shortest_word_path', 'longest_words_raw',
  'longest_word_host', 'longest_word_path', 'avg_words_raw',
  'avg_word_host', 'avg_word_path', 'phish_hints', 'domain_in_brand',
  'brand_in_subdomain', 'brand_in_path', 'suspecious_tld',
  'statistical_report' ]

site_content_columns = ['nb_hyperlinks', 'ratio_intHyperlinks',
  'ratio_extHyperlinks', 'ratio_nullHyperlinks', 'nb_extCSS',
  'ratio_intRedirection', 'ratio_extRedirection', 'ratio_intErrors',
  'ratio_extErrors', 'login_form', 'external_favicon', 'links_in_tags',
  'submit_email', 'ratio_intMedia', 'ratio_extMedia', 'sfh', 'iframe',
  'popup_window', 'safe_anchor', 'onmouseover', 'right_clic',
  'empty_title', 'domain_in_title', 'domain_with_copyright']

third_party_columns = ['whois_registered_domain', 'domain_registration_length', 'domain_age',
       'web_traffic', 'dns_record', 'google_index', 'page_rank']


X_url_train = X_train[url_columns]
X_site_content_train = X_train[site_content_columns]
X_third_party_train = X_train[third_party_columns]

In [50]:
# Recombine X,y training data sets for exploration
Xy_train = X_train.copy()
Xy_train['y'] = y_train

Xy_url_train = X_url_train.copy()
Xy_url_train['y'] = y_train

Xy_site_content_train = X_site_content_train.copy()
Xy_site_content_train['y'] = y_train

Xy_third_party_train = X_third_party_train.copy()
Xy_third_party_train['y'] = y_train

#### Feature-Target Correlation

Third-party features google_index and page_rank dominate the correlations.  However, url features nb_www and ratio_digits_url also look promising, and they're closely followed by site content variables domain_in_title and nb_hyperlinks.


In [47]:
# Calculate correlation matrix
corr_matrix = Xy_train.corr().abs()

# Get top 25 features correlated with the target variable
top_correlated_columns = corr_matrix['y'].sort_values(ascending=False).head(25)
print(top_correlated_columns)

y                      1.000000
google_index           0.738908
page_rank              0.503734
nb_www                 0.434337
ratio_digits_url       0.357516
domain_in_title        0.338552
nb_hyperlinks          0.334818
phish_hints            0.326646
domain_age             0.325361
ip                     0.316615
nb_qm                  0.301241
ratio_intHyperlinks    0.255428
nb_slash               0.246719
length_hostname        0.243607
length_url             0.231634
ratio_digits_host      0.227973
nb_eq                  0.226459
empty_title            0.221384
tld_in_subdomain       0.215112
shortest_word_host     0.210331
prefix_suffix          0.209651
longest_word_path      0.207685
avg_word_path          0.206130
nb_dots                0.203182
ratio_intMedia         0.202347
Name: y, dtype: float64


In [52]:
# Calculate correlation matrix
corr_matrix_url = Xy_url_train.corr().abs()

# Get top 10 url features correlated with the target variable
top_correlated_url_columns = corr_matrix_url['y'].sort_values(ascending=False).head(10)
print(top_correlated_url_columns)

y                    1.000000
nb_www               0.434337
ratio_digits_url     0.357516
phish_hints          0.326646
ip                   0.316615
nb_qm                0.301241
nb_slash             0.246719
length_hostname      0.243607
length_url           0.231634
ratio_digits_host    0.227973
Name: y, dtype: float64


In [54]:
corr_matrix_site_content = Xy_site_content_train.corr().abs()

# Get top 10 url features correlated with the target variable
top_correlated_site_content_columns = corr_matrix_site_content['y'].sort_values(ascending=False).head(10)
print(top_correlated_site_content_columns)

y                        1.000000
domain_in_title          0.338552
nb_hyperlinks            0.334818
ratio_intHyperlinks      0.255428
empty_title              0.221384
ratio_intMedia           0.202347
links_in_tags            0.191689
safe_anchor              0.190779
domain_with_copyright    0.180982
ratio_extRedirection     0.163749
Name: y, dtype: float64


In [55]:
corr_matrix_third_party = Xy_third_party_train.corr().abs()

# Get top 10 url features correlated with the target variable
top_correlated_third_party_columns = corr_matrix_third_party['y'].sort_values(ascending=False).head(10)
print(top_correlated_third_party_columns)

y                             1.000000
google_index                  0.738908
page_rank                     0.503734
domain_age                    0.325361
domain_registration_length    0.170025
dns_record                    0.111166
whois_registered_domain       0.074040
web_traffic                   0.064179
Name: y, dtype: float64


#### Boxplots - TODO - fix or eliminate

A quick (if bulky) look at the distributions the features.  There appear to be some invalid values, mostly -1s in features where these values don't make sense.

In [56]:
#Boxplots
import pandas as pd
import matplotlib.pyplot as plt

# Define the number of columns per plot
columns_per_plot = 12

# Split the numeric columns into chunks
chunks = [Xy_train[i:i + columns_per_plot] for i in range(0, len(Xy_train), columns_per_plot)]

# Create boxplots for each chunk
for i, chunk in enumerate(chunks):
    plt.figure()
    Xy_train[chunk].boxplot()
    plt.title(f'Boxplots for Columns {i * columns_per_plot + 1} to {(i + 1) * columns_per_plot}')
    plt.xticks(rotation=90)  # Rotate x-axis labels for better readability
    plt.show()


ValueError: Boolean array expected for the condition, not object

<Figure size 640x480 with 0 Axes>

#### Parallel coordinate plots - correlated feature groups - TODO: Fix/build out

In [ ]:
#Create empty dataframes
dfs_eda = []
for i in range(0,5):
  dfs_eda.append(pd.DataFrame)

low_corr_threshold = 0.25
high_corr_threshold = 0.45

#
for col in corr_matrix.columns:

  #Get the next most-correlated column other than col itself
  most_correlated = corr_matrix[col].sort_values(ascending=False).index[1]
  correlation = corr_matrix.loc[col, most_correlated]

  for df in dfs_eda:
    if correlation > low_corr_threshold:
      if :
        df[col] = Xy_train[col]
        df[most_correlated] = Xy_train[most_correlated]






In [ ]:
import hiplot as hip

# add y to df1
df['y'] = y_train

# convert df1 to list of dicts because hiplot requires
df1_list = df1.to_dict('records')
hip.Experiment.from_iterable(df1_list).display()

#### Preliminary Modeling - Feature Importance

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import Pipeline
import pandas as pd

# Define model evaluation function

def print_model_evaluation(
    title: str,
    pipe : Pipeline,
    X : pd.DataFrame,
    y : pd.DataFrame):

    print("\n" + title)
    pred_y = pipe.predict(X)
    print(confusion_matrix(pred_y, y))
    print("\nROC-AUC: " + str(roc_auc_score(pred_y, y)))
    print("Precision: " + str(precision_score(pred_y, y)))
    print("Recall: " + str(recall_score(pred_y, y)))

##### Logistic Regression

In [ ]:
#Set up pipeline

import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline

pipe_lr = make_pipeline(
      StandardScaler(),
      LogisticRegression(random_state=42)
)

pipe_lr.fit(X_train, y_train)


In [ ]:
print_model_evaluation("Logistic Regression\nPerformance on Training Set",
                       pipe_lr, X_train, y_train)

print_model_evaluation("Logistic Regression\nPerformance on Validation Set",
                       pipe_lr, X_validation, y_validation)

##### Random Forest

In [ ]:
#Set up & run pipeline - random forest

from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline

pipe_rf = make_pipeline(
      StandardScaler(),
      RandomForestClassifier(random_state=42)
)

pipe_rf.fit(X_train, y_train)


In [ ]:
print_model_evaluation("Random Forest\nPerformance on Training Set",
                       pipe_rf, X_train, y_train)

print_model_evaluation("Random Forest\nPerformance on Validation Set",
                       pipe_rf, X_validation, y_validation)


##### Gradient-boosted Trees

In [ ]:
# Set up and run pipeline - gradient boosted trees

from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.pipeline import make_pipeline

pipe_gbt = make_pipeline(
      StandardScaler(),
      HistGradientBoostingClassifier(random_state=42)
)

pipe_gbt.fit(X_train, y_train)

In [ ]:
print_model_evaluation("Gradient-boosted Trees\nPerformance on Training Set",
                       pipe_gbt, X_train, y_train)

print_model_evaluation("Gradient-boosted Trees\nPerformance on Validation Set",
                       pipe_gbt, X_validation, y_validation)

#### Small Multiples -
heatmaps for something?

### Feature Engineering and Selection

### Preprocessing Development